### Python之简单爬虫——requests、beautifulsoup4

在Python中可以通过requests方便的获取网页内容，然后使用beautifulsoup4解析网页内容，获取想要用到的信息

我们以中国天气网上昆明地区的信息为例来讲述如果获取和解析想要使用的信息，假设我们想要获取的信息包含两块

1. 昆明主城区最近七天的天气情况，主要是日期、天气和温度，如下图所示
![](weather_kunming_latest_7_days.png)

2. 昆明周边地区的情况，主要是地区名和问题，如下图所示
![](weather_kunming_around.png)

这些信息都是在 [http://www.weather.com.cn/weather/101290101.shtml](http://www.weather.com.cn/weather/101290101.shtml) 这一个网页上，所以我们只需要对这一个网页进行处理

首先按照这两个库，使用 `pip install requests beautifulsoup4`，如果安装较慢，也可以使用清华大学的pypi源，使用`pip install -i https://pypi.tuna.tsinghua.edu.cn/simple requests beautifulsoup4`来安装

In [1]:
!pip install requests beautifulsoup4

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


执行上面的命令，安装好这两个库之后，我们就可以开始编写代码进行处理了

##### 第一步，获取网页内容

In [2]:
import requests

page = requests.get("http://www.weather.com.cn/weather/101290101.shtml")
page

<Response [200]>

可以看到使用`request.get("url")`返回的是一个`Response`对象，我们可以查看它都有哪些方法，使用`dir`

In [3]:
dir(page)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

其中最常用的是`status_code`表示获取结果的状态吗，如果是2xx表示成功，4xx/5xx则表示失败，`content`表示获取的内容

In [5]:
# 抓取网页的状态码
page.status_code

# 抓取网页的内容
page.content

200

##### 第二步，现在已经有了网页的html内容了，可以使用beautifulSoup4进行解析处理了

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

可以使用`BeautifulSoup`对象的`prettify`方法格式化html内容

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <link href="http://i.tq121.com.cn" rel="dns-prefetch"/>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   【昆明天气】昆明天气预报,蓝天,蓝天预报,雾霾,雾霾消散,天气预报一周,天气预报15天查询
  </title>
  <meta content="zh-cn" http-equiv="Content-Language"/>
  <meta content="昆明天气预报,昆明今日天气,昆明周末天气,昆明一周天气预报,昆明蓝天,昆明蓝天预报,昆明雾霾,昆明雾霾消散,昆明40日天气预报" name="keywords">
   <meta content="昆明天气预报，及时准确发布中央气象台天气信息，便捷查询昆明今日天气，昆明周末天气，昆明一周天气预报，昆明蓝天预报，昆明天气预报，昆明40日天气预报，还提供昆明的生活指数、健康指数、交通指数、旅游指数，及时发布昆明气象预警信号、各类气象资讯。" name="description"/>
   <meta content="name=天气资讯;action-uri=http://www.weather.com.cn/news/index.shtml;icon-uri=http://www.weather.com.cn/favicon.ico" name="msapplication-task"/>
   <meta content="name=生活天气;action-uri=http://www.weather.com.cn/life/index.shtml;icon-uri=http://www.weather.com.cn/favicon.ico" name="msapplication-task"/>
   <meta content="name=气象科普;action-uri=http://www.weather.com.cn/science/index.shtml;icon-uri=http://www.weather

如何获取想要的信息呢，这时候就需要打开Chrome开发者工具来查看需要用到的数据在html文档的位置，html文档是超文本标记语言的，也就是说它里面的内容是按层次和语义标签嵌套的

![](chrome_devtools_select.png)

一般在html里面会使用css（层叠样式表）中的class或id对特定元素进行定位和设置样式的，所以一般我们使用css的class和id选择器来定位页面中的元素

In [26]:
# 使用 css选择器可参考 https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
# 注意如果有多个css样式的情况处理
#oup.find_all("li", class_="sky skyid")

#print(soup.select("li.sky.skyid"))

print(len(soup.select("li.sky.skyid")))

print(type(soup.select("li.sky.skyid")[0]))

7
<class 'bs4.element.Tag'>


可以看到我们获取了所有的这七天的信息，都是在一个一个li中的，接下来，我们继续提取它里面的日期、天气和温度信息

In [37]:
# 获取日期，注意find_all返回的是一个ResultSet，我们只需要用到第一个，并且使用text属性获取内容
print(soup.select("li.sky.skyid")[0].find_all('h1'))
print(soup.select("li.sky.skyid")[0].find_all('h1')[0].text)

# 类似方式，我们获取天气和温度
print(soup.select("li.sky.skyid")[0].select('p.wea')[0].text)
print(soup.select("li.sky.skyid")[0].select('p.tem')[0].text)

[<h1>27日（今天）</h1>]
27日（今天）
晴

20/4℃



In [39]:
# 所以完整的代码是定义一个列表，用来保存这七天的信息
weather_kunming_7days = []
for day in range(7):
    weather_kunming_7days.append({
        "date": soup.select("li.sky.skyid")[0].find_all('h1')[0].text,
        "weather": soup.select("li.sky.skyid")[0].select('p.wea')[0].text,
        "temperature": soup.select("li.sky.skyid")[0].select('p.tem')[0].text
    })

from pprint import pprint
pprint(weather_kunming_7days)

[{'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'},
 {'date': '27日（今天）', 'temperature': '\n20/4℃\n', 'weather': '晴'}]


接下里我们来获取周边地区的信息，同样使用Chrome开发者工具分析网页结构

![](chrome_devtools_kunming_around.png)

In [48]:
print(len(soup.select("div.aro_city ul li")))
print(soup.select("div.aro_city ul li")[0].select("span")[0].text)
print(soup.select("div.aro_city ul li")[0].select("i")[0].text)

12
晋宁
19/5°C


In [50]:
weather_kunming_around = []
for index in range(len(soup.select("div.aro_city ul li"))):
    liTag = soup.select("div.aro_city ul li")[index]
    weather_kunming_around.append({
        "name": liTag.select("span")[0].text,
        "temperature": liTag.select("i")[0].text
    })

pprint(weather_kunming_around)

[{'name': '晋宁', 'temperature': '19/5°C'},
 {'name': '黔西南', 'temperature': '20/6°C'},
 {'name': '曲靖', 'temperature': '19/3°C'},
 {'name': '寻甸', 'temperature': '20/2°C'},
 {'name': '安宁', 'temperature': '19/2°C'},
 {'name': '禄劝', 'temperature': '20/2°C'},
 {'name': '嵩明', 'temperature': '20/2°C'},
 {'name': '攀枝花', 'temperature': '23/6°C'},
 {'name': '富民', 'temperature': '21/2°C'},
 {'name': '楚雄', 'temperature': '19/6°C'},
 {'name': '呈贡', 'temperature': '18/4°C'},
 {'name': '玉溪', 'temperature': '19/4°C'}]


这样我们就获取了所有想要的信息，之后你就可以进行后续处理

例如整理这里的代码到一个python文件中方便运行、保存信息到文件，我这里找到一个win10 toast弹窗的有意思小类库，可以参考[https://towardsdatascience.com/how-to-make-windows-10-toast-notifications-with-python-fb3c27ae45b9](https://towardsdatascience.com/how-to-make-windows-10-toast-notifications-with-python-fb3c27ae45b9)